In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bsebse/BSE102_Lecture 3_Chemical basis of life.pdf


In [2]:
pip install pymupdf

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install einops flash_attn

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install python-docx

Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import fitz  
from PIL import Image, ImageDraw, ImageFont
import io
from io import BytesIO
import pytesseract  
import torch 
import requests  
from transformers import Blip2Processor, Blip2ForConditionalGeneration, PegasusForConditionalGeneration, PegasusTokenizer, AutoProcessor, AutoModelForCausalLM 
from docx import Document 
from docx.shared import Inches  
import tempfile  
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import BartForConditionalGeneration, BartTokenizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import BartTokenizer
import numpy as np 
import pandas as pd 
nltk.download('punkt')
nltk.download('stopwords')


2024-06-27 17:59:38.936712: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-27 17:59:38.936762: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-27 17:59:38.938170: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large", trust_remote_code=True).to(device)
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large", trust_remote_code=True)

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [9]:
def run_example(image, task_prompt, text_input=None):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input
    inputs = processor(text=prompt, images=image, return_tensors="pt").to(device)
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=1024,
        num_beams=3
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(generated_text, task=task_prompt, image_size=(image.width, image.height))
    caption = parsed_answer["<MORE_DETAILED_CAPTION>"]
    return caption

In [10]:
def pdf_doc(pdf_document):
    doc = fitz.open(pdf_document)

    page_text_list = []
    for page_number in range(len(doc)):
        page = doc.load_page(page_number)  
        image_list = page.get_images(full=True)  

        page_text = page.get_text()
        page_text_combined = page_text

        
        for img_index, img in enumerate(image_list):
            xref = img[0] 
            base_image = doc.extract_image(xref)  
            image_bytes = base_image["image"]
            
            image = Image.open(io.BytesIO(image_bytes)).convert('RGB')
            
            prompt = "<MORE_DETAILED_CAPTION>"
            generated_caption = run_example(image, prompt)

            combined_text = f"\n\nImage {img_index + 1}:\nCaption: {generated_caption}\n"

            page_text_combined += combined_text

        page_text_list.append(page_text_combined)
    return page_text_list

In [11]:
def docx_doc(docx_document):
    doc = Document(docx_document)
    doc_text_list = []
    for para in doc.paragraphs:
        doc_text_list.append(para.text)

        for run in para.runs:
            if run.element.xml.find("pic:blipFill") != -1:
                image = run.element.xpath(".//a:blip")[0]
                image_data = image.get("{http://schemas.openxmlformats.org/officeDocument/2006/relationships}embed")
                image_part = doc.part.related_parts[image_data]
                image_bytes = image_part.blob

                image = Image.open(io.BytesIO(image_bytes)).convert('RGB')
                prompt = "<MORE_DETAILED_CAPTION>"
                generated_caption = run_example(image, prompt)
                combined_text = f"Caption: {generated_caption}\n"
                doc_text_list.append(combined_text)
    return doc_text_list

    



In [12]:
def determine_file_type(file_path):
    _, file_extension = os.path.splitext(file_path)
    if file_extension.lower() == '.docx':
        return 'docx'
    elif file_extension.lower() == '.pdf':
        return 'pdf'
    else:
        return 'unknown'

In [13]:
def get_text(path):
    text_each_page=[]
    file_type=determine_file_type(path)
    if file_type=='pdf':
        text_each_page=pdf_doc(path)
    elif file_type=='docx':
        text_each_page=docx_doc(path)
    else :
        print("invalid file type")
    return text_each_page

In [14]:
def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(tokens)


In [15]:
def process_text_in_sentence_chunks(path, max_chunk_size):
    text=('\n'.join(page for page in get_text(path)))
    sentences = text.replace("\n", " ") 
    sentences = nltk.sent_tokenize(sentences)

    current_chunk = ""
    text_list = []
    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_chunk_size:
            current_chunk += " " + sentence
        else:
              text_list.append(current_chunk.strip())
              current_chunk = sentence

    if current_chunk:
        text_list.append(current_chunk.strip())
    return text_list

# Models****

In [16]:
model_name = 't5-large'
tokenizer1 = T5Tokenizer.from_pretrained(model_name)
model1 = T5ForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1.to(device)

def generate_summary_t5(text, max_length=60):
    preprocess_text = text.strip().replace("\n", " ")
    t5_prepared_text = "summarize: " + preprocess_text

    tokenized_text = tokenizer1.encode(t5_prepared_text, return_tensors="pt", max_length=512, truncation=True)
    tokenized_text = tokenized_text.to(device)
    summary_ids = model1.generate(tokenized_text, max_length=max_length, min_length=5, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer1.decode(summary_ids[0], skip_special_tokens=True)

    return summary

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
model_name = "facebook/bart-large-cnn"
tokenizer2 = BartTokenizer.from_pretrained(model_name)
model2 = BartForConditionalGeneration.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model2.to(device)


def generate_summary_bart(input_text):
    inputs = tokenizer2(input_text, return_tensors="pt", max_length=1024, truncation=True).to("cuda")
    summary_ids = model2.generate(inputs['input_ids'], 
                                 max_length=80, 
                                 min_length=5, 
                                 length_penalty=2.0, 
                                 num_beams=4, 
                                 early_stopping=True)
    summary = tokenizer2.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [18]:
model_name = "google/pegasus-large"
tokenizer3 = PegasusTokenizer.from_pretrained(model_name)
model3 = PegasusForConditionalGeneration.from_pretrained(model_name)


def generate_summary_pegasus(input_text, max_input_length=1024, max_summary_length=60, min_summary_length=15):
    inputs = tokenizer3(input_text, return_tensors="pt", max_length=max_input_length, truncation=True)
    summary_ids = model3.generate(inputs['input_ids'], 
                                 max_length=max_summary_length,
                                 min_length=min_summary_length,  
                                 num_beams=4, 
                                 length_penalty=2.0, 
                                 early_stopping=True)
    summary = tokenizer3.decode(summary_ids[0], skip_special_tokens=True)
    return summary

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
path="/kaggle/input/bsebse/BSE102_Lecture 3_Chemical basis of life.pdf"


In [20]:
input_text=process_text_in_sentence_chunks(path,512)
for text in input_text:
    print(generate_summary_t5(text))

BSE 102 Bioscience Spring 2024 The Chemical Basis of Life Instructor: Dr. sunil Kumar boda Assistant professor department of biosciences & biomedical engineering IIT Indore.
the book is surrounded by a banner with the words "IIT Indore" written in Hindi. below the book, there are two lines of text in Hindi, which translates to "Indian Institute of technology Indore" the emblem is a symbol of knowledge and knowledge
elements are substances that cannot be further broken down to other substances by ordinary chemical means. compounds are substances containing two or more elements combined in a fixed ratio.
2Na + Cl2 2 NaCl; (both Na and Cl are corrosive, but. NaCl is edible!)
major elements in the human body: C, O, H, N, Ca, P (make up 99% of body weight) minor elements ( 1 wt%): K, S, Na, Cl, Mg (for nerve signaling and chemical reactions) trace elements (
the infographic shows a man standing with his back to the viewer. he is wearing a red t-shirt, black shorts, and colorful socks. the i

In [21]:
input_text=process_text_in_sentence_chunks(path,900)
for text in input_text:
    print(generate_summary_bart(text))

The logo is a circular emblem with a blue background and a white border. In the center of the emblem, there is an open book with a sunburst design around it.
Organisms are composed of elements, usually combined into compounds. Matter – occupies space and has mass. Three physical states of matter – solid, liquid and gas. All forms of matter are fundamentally composed of chemical elements.
The elements of the human body are C, O, H, N, Ca, P (make up 99% of body weight) Minor elements: K, S, Na, Cl, Mg (for nerve signaling and chemical reactions) Trace elements: B, Cr, Cu, F, I, Fe, Mn, Mo, Se, Si, Sn, V, Zn
He is standing with one leg slightly bent at the knee and the other leg extended behind him. The infographic is divided into three sections. The chart also includes information about the different elements and their properties.
Iodine deficiency leads to abnormal swelling of the thyroid gland leading to a condition called goiter. Iron deficiency anemia occurs due to lack of iron to p

In [23]:
input_text=process_text_in_sentence_chunks(path,512)
for text in input_text:
    print(generate_summary_pegasus(text))

It is a circular emblem with a blue background and a white border.
Below the book, there are two lines of text in Hindi, which translates to "Indian Institute of technology Indore".
Organisms are composed of elements, usually combined into compounds Matter – occupies space and has mass Three physical states of matter – solid, liquid and gas All forms of matter – rocks, water, air and biological systems are fundamentally composed of chemical elements.
Eg: 2Na + Cl2 2 NaCl; (Both Na and Cl are corrosive, but NaCl is edible!)
2H2 + O2 2H2O Compounds in living organisms comprise of 3 or 4 elements: • Carbohydrates: Carbon (C), Oxygen (O), Hydrogen (H) • Proteins: Carbon (C), Oxygen (O), Hydrogen (H), Nitrogen
It shows a silhouette of a man standing with his back to the viewer, facing the viewer.
Trace elements are common additives to food and water Iodine is an essential component of the hormones thyroxine (T4 - inactive) and triiodothyronine (T3 - active) produced by the thyroid glands.
I